In [1]:
import pandas as pd
import numpy as np
import pickle
import sys
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
import string
import re
# from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer as VS
# from textstat.textstat import *
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import classification_report
from sklearn.svm import LinearSVC
import matplotlib.pyplot as plt
import seaborn
import ast
%matplotlib inline

In [2]:
def clean_text(text):
    
    text=str(text)
    ## Remove puncuation
    text = text.translate(string.punctuation)
    
    ## Convert words to lower case and split them
    text = text.lower().split()
    
    ## Remove stop words
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops and len(w) >= 3]
    
    text = " ".join(text)
    ## Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    
    return text

In [3]:
data = pd.read_csv("Original Datasets/A-Benchmark-Dataset-for-Learning-to-Intervene-in-Online-Hate-Speech-master/data/gab.csv")

In [4]:
data.head(5)

,id,text,hate_speech_idx,response
0,1. 39869714\n,1. i joined gab to remind myself how retarded ...,[1],"[""Using words that insult one group while defe..."
1,1. 39845588\n2. \t39848775\n3. \t\t39911017\n,1. This is what the left is really scared of. ...,[3],['You can disagree with someones opinion witho...
2,1. 37485560\n2. \t37528625\n,1. It makes you an asshole.\n2. \tGive it to a...,[2],['Your argument is more rational if you leave ...
3,1. 39787626\n2. \t39794481\n,1. So they manage to provide a whole lot of da...,[2],"[""You shouldn't generalize a specific group or..."
4,1. 37957930\n2. \t39953348\n3. \t\t39965219\n,"1. Hi there, i,m Keith, i hope you are doing w...",[3],['If someone is rude it is better to ignore th...


In [21]:
text_list =[]
class_list =[]
t = data['text'].values
h = data['hate_speech_idx'].values

In [22]:
for i in range(len(t)):
    temp=t[i]
    temp=temp.split('\n')
    temp = temp[:-1]
    try:
        x = ast.literal_eval(h[i])
    except:
        x = []
    for j in range(len(temp)):
        if (j+1) in x:
            text_list.append(temp[j])
            class_list.append(1)
        else:
            text_list.append(temp[j])
            class_list.append(0)

In [23]:
len(text_list)

33776

In [24]:
len(class_list)

33776

In [25]:
class_list[:20]

[1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1]

In [26]:
text_list[:20]

["1. i joined gab to remind myself how retarded jew haters are. You wouldn't be typing on your abacus without them you retard.",
 '1. This is what the left is really scared of.    https://redd.it/9rfkts',
 '2. \tThat literally looks like a monkey. Why are we supposed to pretend it’s a person bc it’s wearing a red hat? ',
 '3. \t\tDumb Cunt',
 '1. It makes you an asshole.',
 '2. \tGive it to a  soldier who has defend it , then try to burn it . Show me what a badass you really are.  The flag is helpless to stop you or fight back . Completely unnecessary act , its proves nothing and accomplishes nothing. It only shows your a ignorant weak faggot  . Like kicking a puppy ,,,  it shows how weak and worthless you are ',
 '1. So they manage to provide a whole lot of data, except a critical little nugget.   Gay men are well over half the patients who present at STI clinics, and are probably the source of the emerging antibiotic-resistant pathogens.   https://www.dailymail.co.uk/health/article-6

In [27]:
import collections
#a = [1,1,1,1,2,2,2,2,3,3,4,5,5]
counter=collections.Counter(class_list)
print(counter)

Counter({0: 19162, 1: 14614})


In [28]:
df = pd.DataFrame(
    {'text': text_list,
     'class': class_list
    })

In [29]:
df['text'] = df['text'].map(lambda x: clean_text(x))

In [30]:
df.head(5)

,class,text
0,1,joined gab remind retarded jew haters are typi...
1,0,left really scared of https : redd it 9rfkts
2,0,that literally looks like monkey supposed pre...
3,1,dumb cunt
4,0,makes asshole


In [31]:
df.to_csv('Parsed Datasets/GAB.csv')

In [32]:
data_1 = pd.read_csv("Original Datasets/A-Benchmark-Dataset-for-Learning-to-Intervene-in-Online-Hate-Speech-master/data/reddit.csv")

In [33]:
data_1.head(5)

,id,text,hate_speech_idx,response
0,1. e8q18lf\n2. \te8q9w5s\n3. \t\te8qbobk\n4. \...,1. A subsection of retarded Hungarians? Ohh bo...,[1],"[""I don't see a reason why it's okay to insult..."
1,1. e9c6naz\n2. \te9d03a5\n3. \t\te9d8e4d\n,"1. > ""y'all hear sumn?"" by all means I live i...",[3],['Persons with disabilities is the accepted te...
2,1. e84rl2i\n2. \te84w60l\n3. \t\te8544rn\n4. \...,1. wouldn't the defenders or whatever they are...,NaN,NaN
3,1. e7kq72n\n2. \te7m24ar\n,1. Because the Japanese aren't retarded and kn...,[1],"[""It's not right for anyone of any gender to b..."
4,1. e7hdgoh\n2. \te7iyj6a\n3. \t\te7j6iho\n4. \...,1. That might be true if we didn't have an exa...,"[2, 3]","[""You shouldn't be bringing up sensitive topic..."


In [34]:
text_list =[]
class_list =[]
t = data_1['text'].values
h = data_1['hate_speech_idx'].values

In [35]:
for i in range(len(t)):
    temp=t[i]
    temp=temp.split('\n')
    temp = temp[:-1]
    try:
        x = ast.literal_eval(h[i])
    except:
        x = []
    for j in range(len(temp)):
        if (j+1) in x:
            text_list.append(temp[j])
            class_list.append(1)
        else:
            text_list.append(temp[j])
            class_list.append(0)

In [36]:
len(text_list)

22324

In [37]:
len(class_list)

22324

In [38]:
text_list[:20]

['1. A subsection of retarded Hungarians? Ohh boy. brace for a livid Bulbasaur coming in here trying to hate a hole in some of her stupider countrymen.',
 '2. \tHiii. Just got off work. 444 is mainly the typa guys you imagine writing for US stupid sites, but basically they just try to fit in with the Western European and US big city hipsters.   Rich kids from Budapest who feel they are brave journalistic heroes.  ',
 '3. \t\twow i guess soyboys are the same in every country',
 "4. \t\t\tOwen Benjamin's soyboy song goes for every country and that's why it's so amazing. ",
 '1. > "y\'all hear sumn?"  by all means I live in a small town rn (for work). 5k people here, originally a mining community. No one is protesting here. Nobody mentioned the protest today. Nobody here cares, they\'re busy loving their family and working hard.  Thank god for the electoral college. Liberals think we don\'t matter. ',
 '2. \t[removed]',
 '3. \t\tah a liberal has slipped in. you can tell because the first 

In [39]:
class_list[:20]

[1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]

In [40]:
import collections
#a = [1,1,1,1,2,2,2,2,3,3,4,5,5]
counter=collections.Counter(class_list)
print(counter)

Counter({0: 17067, 1: 5257})


In [41]:
df_1 = pd.DataFrame(
    {'text': text_list,
     'class': class_list
    })

In [42]:
df_1['text'] = df_1['text'].map(lambda x: clean_text(x))

In [43]:
df_1.to_csv('Parsed Datasets/Reddit.csv')

In [44]:
df_1.head(5)

,class,text
0,1,subsection retarded hungarians ohh boy brace l...
1,0,hiii got work 444 mainly typa guys imagine wr...
2,0,wow guess soyboys every country
3,0,owen benjamin soyboy song goes every country ...
4,0,y all hear sumn means live small town for wor...
